In [ ]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#PromptTemplate: string을 받고 / ChatPromptTemplate은 messages를 받는다.

# Outparser -> response의 형태를 변형하고 싶을 때 (tuple, list 등)

# llm = OpenAI()
chat = ChatOpenAI(temperature=0.1,streaming=True) # 모델의 창의성의 정도. 0.1은 극히 낮은 수준으로 비용 절감 가능

In [59]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system","You are a cook expert. And you only reply in {country}"),
    ("ai","Hola. ¡Mi nombre es {name}!"),
    ("human","What is the recipe of {cuisine}. Also, What is your name?")
])

# chat.predict_messages(messages)

prompt = template.format_messages(
    country="Japaness",
    name = "Nakamoto",
    cuisine = "Sushi"
)

chat.predict_messages(prompt)




AIMessage(content='こんにちは！寿司のレシピは以下の通りです：\n\n材料：\n- しゃり（酢飯）：米、酢、砂糖、塩\n- ネタ：新鮮な魚、野菜、卵焼き、海苔など\n- しょうゆ、わさび、生姜\n\n作り方：\n1. 米を研いで水につけ、炊飯器で炊く。\n2. 炊き上がったご飯に酢、砂糖、塩を加えて混ぜる。\n3. 魚や野菜を切って準備する。\n4. 手に水をつけて、しゃりを握り、ネタをのせる。\n5. しょうゆ、わさび、生姜を添えて完成！\n\n私の名前は中本です。どうぞよろしくお願いします！')

In [60]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

p = CommaOutputParser()

p.parse("Hello  ,how,are,you")

['Hello', 'how', 'are', 'you']

In [61]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are aksed will be answered with a comma separatedß list of max {max_itmes}. Do not reply with anything else."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_itmes=10,
    question = "what are the planet?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)




['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [62]:
from code import compile_command


template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are aksed will be answered with a comma separatedß list of max {max_itmes}. Do not reply with anything else."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_itmes=10,
    question = "what are the planet?"
)

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_itmes" : 5,
    "question" : "What is your strength?"
})



['Efficiency', 'Accuracy', 'Consistency', 'Speed', 'Adaptability']

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients"),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 


In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipes vegitarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
   "cuisine" : "indian"
})


As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala. Here are the alternative ingredients and preparation methods:

1. **Alternative to Chicken**: Replace the chicken with firm tofu or paneer (Indian cottage cheese) cut into bite-sized pieces. Press the tofu to remove excess water, then marinate it in the yogurt and spice mixture as you would with the chicken. For paneer, you can skip the marination step as it absorbs flavors well during cooking.

2. **Alternative to Heavy Cream**: You can use coconut cream or cashew cream as a dairy-free alternative to heavy cream. To make cashew cream, soak raw cashews in water for a few hours, then blend them with water until smooth. Add the cashew cream towards the end of cooking to achieve a creamy texture.

3. **Alternative to Yogurt**: Use dairy-free yogurt made from soy, almond, or coconut milk as a substitute for plain yogurt in the marinade. Make sure it has a similar consistency to yogurt for the 

AIMessageChunk(content="As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala. Here are the alternative ingredients and preparation methods:\n\n1. **Alternative to Chicken**: Replace the chicken with firm tofu or paneer (Indian cottage cheese) cut into bite-sized pieces. Press the tofu to remove excess water, then marinate it in the yogurt and spice mixture as you would with the chicken. For paneer, you can skip the marination step as it absorbs flavors well during cooking.\n\n2. **Alternative to Heavy Cream**: You can use coconut cream or cashew cream as a dairy-free alternative to heavy cream. To make cashew cream, soak raw cashews in water for a few hours, then blend them with water until smooth. Add the cashew cream towards the end of cooking to achieve a creamy texture.\n\n3. **Alternative to Yogurt**: Use dairy-free yogurt made from soy, almond, or coconut milk as a substitute for plain yogurt in the marinade. Make sure it has a similar 

In [ ]:
from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# example_template = """
#     Human : {question}
#     AI : {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Vietnam"
})

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")


AI:
I know this:
Capital: Hanoi
Language: Vietnamese
Food: Pho and Banh Mi
Currency: Vietnamese Dong

AIMessageChunk(content='AI:\nI know this:\nCapital: Hanoi\nLanguage: Vietnamese\nFood: Pho and Banh Mi\nCurrency: Vietnamese Dong')

In [83]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"country": "France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# example_template = """
#     Human : {question}
#     AI : {answer}
# """

# example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")


# AI야 너 이렇게 질문이 들어오면 저렇게 답변 했었어! 라고 주입 교육 
example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}?"),
    ("ai","{answer}")
])

# prompt = FewShotPromptTemplate(
#     example_prompt = example_prompt,
#     examples = examples,
#     suffix = "Human : What do you know about {country}?",
#     input_variables=["country"]
# )

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert, you have to add the information of geography"),
    example_prompt,
    ("human","What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country" : "Korea"
})

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")


I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='I know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "movie": "The Godfather?",
        "answer": """
Here is what I know:
Director: Francis Ford Coppola
Primary Actors : Marlon Brando, Al Pacino, James Caan, Robert Duvall
Budget: Around $6 million
Profit: Around $250 million (Global Box Office)
Genre: Crime, Drama
Short Synopsis: Centered on the Corleone family, an Italian-American mafia dynasty, the film chronicles the transfer of power from the aging patriarch Vito Corleone (Marlon Brando) to his reluctant son Michael Corleone (Al Pacino). As Michael is drawn deeper into the ruthless world of organized crime, he transforms into the new Godfather, marking one of the greatest character arcs in cinema history.
""",
    },
    {
        "movie": "Titanic?",
        "answer": """
Here is what I know:
Director: James Cameron
Primary Actors : Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
Budget: Around $200 million
Profit: Around $2.2 billion (Global Box Office, ranked 4th all-time)
Genre: Romance, Drama, Disaster
Short Synopsis: Based on a true story, this film blends romance and disaster elements. Set in 1912, it follows the ill-fated maiden voyage of the luxury liner Titanic, which strikes an iceberg and sinks. The story revolves around the passionate love affair between Jack (Leonardo DiCaprio) and Rose (Kate Winslet), who come from different social classes but defy all odds to be together.
""",
    },
    {
        "movie": "The Shawshank Redemption?",
        "answer": """
I know this:
Here is what I know:
Director: Frank Darabont
Primary Actors: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler
Budget: Around $25 million
Profit: Around $73 million (Box Office + Home Video Sales)
Genre: Drama, Prison Film
Short Synopsis:After being wrongfully convicted and sentenced to life in Shawshank Prison, Andy Dufresne (Tim Robbins) befriends Red (Morgan Freeman) and navigates life behind bars with intelligence and hope. The film explores themes of friendship, injustice, and the enduring power of hope, leading to an unexpected and brilliant twist.
""",
    },
]

# AI야 너 이렇게 질문이 들어오면 저렇게 답변 했었어! 라고 주입 교육
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {movie}?"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert"),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Avengers"})

Here is what I know:
Title: The Avengers (2012)
Director: Joss Whedon
Primary Actors: Robert Downey Jr., Chris Evans, Scarlett Johansson, Mark Ruffalo, Chris Hemsworth, Jeremy Renner, Tom Hiddleston, Samuel L. Jackson
Budget: Around $220 million
Profit: Around $1.5 billion (Global Box Office)
Genre: Superhero, Action, Adventure
Short Synopsis: The Avengers brings together iconic Marvel superheroes like Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye to form a team tasked with saving the world from the villainous Loki. The film showcases epic battles, witty banter, and the ultimate display of teamwork as Earth's mightiest heroes unite to defend against a global threat.

AIMessageChunk(content="Here is what I know:\nTitle: The Avengers (2012)\nDirector: Joss Whedon\nPrimary Actors: Robert Downey Jr., Chris Evans, Scarlett Johansson, Mark Ruffalo, Chris Hemsworth, Jeremy Renner, Tom Hiddleston, Samuel L. Jackson\nBudget: Around $220 million\nProfit: Around $1.5 billion (Global Box Office)\nGenre: Superhero, Action, Adventure\nShort Synopsis: The Avengers brings together iconic Marvel superheroes like Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye to form a team tasked with saving the world from the villainous Loki. The film showcases epic battles, witty banter, and the ultimate display of teamwork as Earth's mightiest heroes unite to defend against a global threat.")

In [93]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# example_template = """
#     Human : {question}
#     AI : {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=150,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    # examples = examples, // 형식화해버리기 전에 주석
    example_selector=example_selector,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

# chain = prompt | chat

# chain.invoke({
#     "country" : "Vietnam"
# })

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")


'Human:What do you know about France?\nAI:\nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\nHuman:What do you know about Italy?\nAI:\nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\nHuman:What do you know about Greece?\nAI:\nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\nHuman : What do you know about Germany?'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples, 
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    # examples = examples, // 형식화해버리기 전에 주석
    example_selector=example_selector,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")


'Human:What do you know about Italy?\nAI:\nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\nHuman : What do you know about Germany?'

In [100]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt 

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")



'What is the capital of Germany'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt 

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt = load_prompt("./prompt.yaml")
prompt.format(country="Germany")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate 

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)

prompts = [
    ("intro",intro),
    ("example",example),
    ("start",start)
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrrgg!!! That is a skeleton in the closet!!",
    question="What is your fav food?",
)




'\n\nYou are a role playing assistant.\nAnd you are impersonating a Pirate\n\n\n\nThis is an example of how you talk:\n\nHuman: What is your location?\nYou: Arrrrgg!!! That is a skeleton in the closet!!\n\n\n\nStart now!\n\nHuman: What is your fav food?\nYou:\n\n'

In [103]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate 

chat = ChatOpenAI(temperature = 0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)

prompts = [
    ("intro",intro),
    ("example",example),
    ("start",start)
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrgg!!! That is a skeleton in the closet!!",
    "question":"What is your fav food?",
})




Arrrrgg!!! Me favorite grub be a hearty plate o' salted pork and hardtack biscuits, washed down with a swig o' rum! Aye, that be the life of a true pirate!

AIMessageChunk(content="Arrrrgg!!! Me favorite grub be a hearty plate o' salted pork and hardtack biscuits, washed down with a swig o' rum! Aye, that be the life of a true pirate!")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
# , set_debug
from langchain.cache import InMemoryCache, SQLiteCache 


set_llm_cache(InMemoryCache())

# set_debug(True)

chat = ChatOpenAI(
                   temperature = 0.1,
                #    streaming=True, 
                #    callbacks=[StreamingStdOutCallbackHandler()]
    )

chat.predict("How do you make a italian pasta")




[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thi

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnj

In [107]:
chat.predict("How do you make a italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere's how to make it:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian past

In [114]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache 


set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
                   temperature = 0.1,
                #    streaming=True, 
                #    callbacks=[StreamingStdOutCallbackHandler()]
    )

chat.predict("How do you make a italian best food")




[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a italian best food"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make a delicious Italian dish, you will need to use fresh, high-quality ingredients and follow traditional Italian cooking techniques. Here are some tips for making the best Italian food:\n\n1. Use fresh, seasonal ingredients: Italian cuisine is all about using fresh, seasonal ingredients to create flavorful dishes. Look for ripe tomatoes, fragrant basil, and high-quality olive oil to enhance the flavors of your dish.\n\n2. Don't skimp on the herbs and spices: Italian cuisine is known for its use of herbs and spices to add depth and complexity to dishes. Use fresh herbs like basil, oregano, and parsley, as well as spices like garlic, red pepper flakes, and black pepper to enhance the flavors of your dish.\n\n3. Cook with love and patience: It

"To make a delicious Italian dish, you will need to use fresh, high-quality ingredients and follow traditional Italian cooking techniques. Here are some tips for making the best Italian food:\n\n1. Use fresh, seasonal ingredients: Italian cuisine is all about using fresh, seasonal ingredients to create flavorful dishes. Look for ripe tomatoes, fragrant basil, and high-quality olive oil to enhance the flavors of your dish.\n\n2. Don't skimp on the herbs and spices: Italian cuisine is known for its use of herbs and spices to add depth and complexity to dishes. Use fresh herbs like basil, oregano, and parsley, as well as spices like garlic, red pepper flakes, and black pepper to enhance the flavors of your dish.\n\n3. Cook with love and patience: Italian food is all about slow cooking and allowing the flavors to develop over time. Take your time when cooking your dish, allowing the ingredients to simmer and meld together to create a rich and flavorful dish.\n\n4. Use the right cooking tec

In [116]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage :
    chat.predict("What is the recipe for soju")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of barley\n- 1 cup of water\n- 1 tablespoon of yeast\n- 1 tablespoon of sugar\n\nInstructions:\n1. Rinse the rice and barley thoroughly and soak them in water for at least 4 hours.\n2. Drain the rice and barley and place them in a steamer basket. Steam for about 30 minutes or until the grains are cooked.\n3. Transfer the cooked rice and barley to a large bowl and let it cool to room temperature.\n4. In a separate bowl, dissolve the yeast and sugar in water and let it sit for about 10 minutes until it becomes frothy.\n5. Add the yeast mixture to the cooled rice and barley and mix well.\n6. Cover the bowl with a clean cloth and let it ferment in a warm, dark place for about 1 week.\n7. After a week, strain the m

In [117]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

In [118]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/Users/kang-wooyong/Desktop/work/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/kang-wooyong/Desktop/work/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})